<a href="https://colab.research.google.com/github/jeyasubha-26/smartid/blob/main/smartId.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install deepface pandas opencv-python-headless scikit-learn

from sklearn.datasets import fetch_lfw_people
import pandas as pd
from deepface import DeepFace
from datetime import datetime
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

# Step 1: Load dataset from sklearn (auto-downloads & caches)
print("📥 Loading LFW dataset from sklearn...")
lfw_people = fetch_lfw_people(min_faces_per_person=20, resize=0.5)
print("✅ Dataset loaded:", lfw_people.images.shape)

# Step 2: Create folder structure like normal dataset
import os
import shutil

base_path = "lfw_dataset"
if not os.path.exists(base_path):
    os.makedirs(base_path)
    for img, person in zip(lfw_people.images, lfw_people.target):
        person_name = lfw_people.target_names[person]
        person_folder = os.path.join(base_path, person_name)
        os.makedirs(person_folder, exist_ok=True)
        img_path = os.path.join(person_folder, f"{len(os.listdir(person_folder))}.jpg")
        from PIL import Image
        Image.fromarray((img * 255).astype("uint8")).save(img_path)

# Step 3: Pick a test image
test_img = os.path.join(base_path, lfw_people.target_names[0], os.listdir(os.path.join(base_path, lfw_people.target_names[0]))[0])

attendance = []

# Step 4: Face matching
for person in os.listdir(base_path):
    person_folder = os.path.join(base_path, person)
    ref_img = os.path.join(person_folder, os.listdir(person_folder)[0])
    try:
        result = DeepFace.verify(test_img, ref_img, model_name='Facenet')
        if result["verified"]:
            print(f"✅ Present: {person}")
            attendance.append({"Name": person, "Status": "Present", "Time": datetime.now().strftime("%H:%M:%S")})
            break
    except Exception as e:
        print(f"⚠️ Error checking {person}: {e}")

# Mark absentees
for person in os.listdir(base_path):
    if not any(a["Name"] == person for a in attendance):
        attendance.append({"Name": person, "Status": "Absent", "Time": "-"})

# Step 5: Save CSV
df = pd.DataFrame(attendance)
df.to_csv("attendance.csv", index=False)
print("\n📄 Attendance saved to attendance.csv")
print(df)

# Step 6: Email absentees
absentees = df[df["Status"] == "Absent"]["Name"].tolist()
absent_text = "\n".join(absentees) if absentees else "No absentees today."

sender_email = "kit27.ad027@gmail.com"
receiver_email = "jeyasubhaganesan2006@gmail.com"
password = "ikienxbatvlzaxxu"  # Your app password

subject = "Daily Absentee List"
body = f"Hello,\n\nThe following students were absent today:\n{absent_text}\n\nRegards,\nSmart ID Attendance System"

msg = MIMEMultipart()
msg["From"] = sender_email
msg["To"] = receiver_email
msg["Subject"] = subject
msg.attach(MIMEText(body, "plain"))

try:
    with smtplib.SMTP_SSL("smtp.gmail.com", 465) as server:
        server.login(sender_email, password)
        server.sendmail(sender_email, receiver_email, msg.as_string())
    print(f"📧 Email sent successfully to {receiver_email}")
except Exception as e:
    print("❌ Email sending failed:", e)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.3/128.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.9/115.9 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.0 MB/s eta 0:00:00
25-09-25 13:58:11 - Directory /root/.deepface has been created
25-09-25 13:58:11 - Directory /root/.deepface/weights has been created
📥 Loading LFW dataset from sklearn...
✅ Dataset loaded: (3023, 62, 47)


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/facenet_weights.h5
To: /root/.deepface/weights/facenet_weights.h5


25-09-25 13:58:44 - 🔗 facenet_weights.h5 will be downloaded from https://github.com/serengil/deepface_models/releases/download/v1.0/facenet_weights.h5 to /root/.deepface/weights/facenet_weights.h5...


100%|██████████| 92.2M/92.2M [00:00<00:00, 223MB/s]


⚠️ Error checking Ricardo Lagos: Exception while processing img2_path
✅ Present: Alejandro Toledo

📄 Attendance saved to attendance.csv
                     Name   Status      Time
0        Alejandro Toledo  Present  13:59:35
1   Megawati Sukarnoputri   Absent         -
2         John Negroponte   Absent         -
3            Alvaro Uribe   Absent         -
4       Jeremy Greenstock   Absent         -
..                    ...      ...       ...
57        Serena Williams   Absent         -
58          Ricardo Lagos   Absent         -
59         Saddam Hussein   Absent         -
60      Lindsay Davenport   Absent         -
61            Tiger Woods   Absent         -

[62 rows x 3 columns]
📧 Email sent successfully to jeyasubhaganesan2006@gmail.com
